get_image# **Image Optimization & DB 2019**
## **1 Adding the Unique 2019 DataBases**
```
GET Text Data
```

In [1]:
import pandas as pd
import requests, re
from lxml.html import fromstring, tostring
from arts import get_image_urls

table_artist = pd.read_csv('./data/artist_2019.csv', header=0).fillna('')
table_columns = table_artist.columns.to_list()
table_columns[0] = 'code'
table_artist.columns = table_columns
artist_code = {table_artist.loc[_,'name'] :table_artist.loc[_,'code']  for _ in range(len(table_artist))}
table_artist.head(3)

,code,name,profile,instagram,facebook,naver,blog
0,A01,강인태,"가족, 디자인, 장난감, 여행을 좋아하는 그래픽 디자이너입니다.",,,,
1,A02,김민지,다양한 매체를 융합한 설치와 기획을 하는 시각예술가 김민지 입니다.,https://www.instagram.com/minjikim_artist/,,,
2,A03,박진호,창의와 통찰에 관심이 많은 저는 남들과 같은 것을 보고 다른생각을 하는 것을 중요하...,https://www.instagram.com/_chamjin_/,,https://blog.naver.com/malangmalang_book,https://brunch.co.kr/@chamjin


In [2]:
# Read the DataFrame & Adding the Artist Code
table_store = pd.read_csv('./data/2019_store.csv', header=0)
table_store.insert(0,'code_artist', [artist_code[_]  for _ in table_store['artist']])

# Adding the Store Code based on Artist Code
store_code = []
for code_artist in sorted(set([_ for _ in table_store['code_artist']])):
    for no,_ in enumerate(table_store[table_store['code_artist']== code_artist].index.to_list()):
        store_code.append(f"{code_artist}{no+1}")
table_store.insert(2, 'code_store', store_code)
table_store.head(2)

,code_artist,artist,code_store,title,address,phone,time,page,problem,1.문제점도출,2.아이데이션및스케치,3.디자인스케치
0,A01,강인태,A011,군산식당,서울특별시 관악구 당곡길 46,010-3350-1266,월~토 오전,1,-기존 오래된 카탈로그,기존 카탈로그가 오래되어 현재 취급품목 소개 위주의 디자인 리뉴얼 필요,현재 정육업계의 트렌드 분석과 카탈로그 디자인 리서치,현재 취급 품목을 새롭게 촬영하고 전문적인 정육업체의 분위기로 디자인 리뉴얼
1,A01,강인태,A012,굿모닝보청기,서울특별시 관악구 신림동 1422-2,02-882-0670,"오전 9시 ~ 오후 18시 (일요일, 공휴일 예약상담)",96,주고객층인 노령자가 해당업체를 찾아오는 과정이 어렵고 눈에 잘 띄지 않음\n입점 업...,업체들이 경쟁적으로 붙인 안내판으로 산만해진 입구,입점업체를 한눈에 보기 쉽도록 정리 필요,요청업체의 브랜드 아이덴티티를 살리면서 전체 입점업체의 정보를 한눈에 볼 수 있도록...


## **2 Getting the Store Images**
Store 마다 저장된 이미지 자료 가져오기

In [3]:
from arts import get_image_urls, change_file_name
from tqdm import tqdm
main_image_index_list = ["banner", "before", "img1", "img2", "img3", "after", "map"]

for _ in tqdm(range(len(table_store))):
    instance = table_store.iloc[_,:]
    code = instance.code_store
    page_index = int(instance['page'])
    # link_list  = get_image_urls(page_index)
    
#     for no, link in enumerate(link_list['main']):
#         file_name = f"{code}_{main_image_index_list[no]}"
#         change_file_name(link, file_name)
    
#     for no, link in enumerate(link_list['after']):
#         file_name = f"{code}_process{no:02d}"
#         change_file_name(link, file_name)

100%|██████████| 54/54 [00:00<00:00, 1165.59it/s]


## **3 Getting the Store Images**
Store 마다 저장된 이미지 자료 가져오기

In [4]:
from arts import naver_address
result = []
for _ in tqdm(table_store['address'].to_list()):
    result.append(naver_address(_)[::-1])

100%|██████████| 54/54 [00:07<00:00,  7.69it/s]


In [5]:
for no, _col in enumerate(['Latitude', 'Longitude', 'address_eng', 'address_old', 'address_road']):
    data = [_[no]   for _ in result]
    table_store.insert(5, _col, data)

# table_store.to_csv('store2019full.csv', index=None, header=None)
table_store.head(2)

,code_artist,artist,code_store,title,address,address_road,address_old,address_eng,Longitude,Latitude,phone,time,page,problem,1.문제점도출,2.아이데이션및스케치,3.디자인스케치
0,A01,강인태,A011,군산식당,서울특별시 관악구 당곡길 46,서울특별시 관악구 당곡길 46,서울특별시 관악구 봉천동 710-6,"46, Danggok-gil, Gwanak-gu, Seoul, Republic of...",126.9285113,37.4911554,010-3350-1266,월~토 오전,1,-기존 오래된 카탈로그,기존 카탈로그가 오래되어 현재 취급품목 소개 위주의 디자인 리뉴얼 필요,현재 정육업계의 트렌드 분석과 카탈로그 디자인 리서치,현재 취급 품목을 새롭게 촬영하고 전문적인 정육업체의 분위기로 디자인 리뉴얼
1,A01,강인태,A012,굿모닝보청기,서울특별시 관악구 신림동 1422-2,서울특별시 관악구 신림로64길 46 휴드림,서울특별시 관악구 신림동 1422-2 휴드림,"46, Sillim-ro 64-gil, Gwanak-gu, Seoul, Republ...",126.9322618,37.4851169,02-882-0670,"오전 9시 ~ 오후 18시 (일요일, 공휴일 예약상담)",96,주고객층인 노령자가 해당업체를 찾아오는 과정이 어렵고 눈에 잘 띄지 않음\n입점 업...,업체들이 경쟁적으로 붙인 안내판으로 산만해진 입구,입점업체를 한눈에 보기 쉽도록 정리 필요,요청업체의 브랜드 아이덴티티를 살리면서 전체 입점업체의 정보를 한눈에 볼 수 있도록...
